<a href="https://colab.research.google.com/github/Wesley-Janson/transformers_for_human_vs_ai_text_identification/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import relevant packages and data_loader.py
#import data_loader

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d, MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split

In [2]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd

In [3]:
def load_data(csv):
  # Reads the raw csv file and split into
  # sentences (x) and target (y)
  df = pd.read_csv(csv)  
  text = df['intro'].values
  labels = df['type'].values
  return labels,text


In [4]:

# This function processes training data, establishing number IDs for each vocabulary word,
# converting word sequence into ID sequence (input_as_ids), and providing dict
# to map from word to its ID (word2id), and list to map from ID back to word (id2word)
def process_training_data(corpus_text):
        """Tokenizes a text file."""
        # Create the model's vocabulary and map to unique indices
        word2id = {}
        id2word = []
        indexes_dropped = []
        list_of_inputs = []
        for j, entry in enumerate(corpus_text):
            for i,word in enumerate(entry):
                if 7<i<=30:
                    if word not in word2id:
                        id2word.append(word)
                        word2id[word] = len(id2word) - 1

            # Convert string of text into string of IDs in a tensor for input to model
            input_as_ids = []
            for i,word in enumerate(entry):
                if 7<i<=30:
                    input_as_ids.append(word2id[word])
            if len(input_as_ids) == 23:
              list_of_inputs.append(input_as_ids)
            else:
              indexes_dropped.append(j)
            # final_ids = torch.LongTensor(input_as_ids)
        list_of_inputs = torch.Tensor(list_of_inputs)

        return list_of_inputs,word2id,id2word, indexes_dropped

In [8]:
# Run data loader
labels, text = load_data('data.csv')
small_labels = labels[:10000]
small_text = text[:10000]
train_x, val_x, train_y, val_y = train_test_split(small_text, small_labels, test_size = 0.2)
print(train_y)
print(val_y)
tokenizer = get_tokenizer("basic_english")
print('now for inputs')
print(train_x)
print(val_x)
list_of_tokens_train = [tokenizer(x) for x in train_x]
list_of_tokens_val = [tokenizer(x) for x in val_x]

train_x,word2id,id2word, indexes_dropped_train = process_training_data(list_of_tokens_train)

val_x,word2id,id2word, indexes_dropped_val = process_training_data(list_of_tokens_val)


[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
now for inputs
['Grayson Murphy (born February 4, 1999) is an American professional basketball player for the Philadelphia 76ers of the National Basketball Association (NBA). He played college basketball for Duke.\n\nMurphy attended Hillcrest High School in Raleigh, North Carolina. As a junior in 2015–16, he averaged 21.4 points, 10.4 rebounds, 3.1 assists and 1.6 blocks per game while leading Hillcrest to a 28–3 record and the North Carolina Class 5A state championship. As a senior in 2016–17, Murphy averaged 25.2 points, 11.9 rebounds, 3.8 assists and 1.8 blocks per game while helping Hillcrest to a 31–1 record and the North Carolina Class 5A state championship.\n\nMurphy was one of the most highly touted high school players in the country and was ranked as the No. 2 player in the Class of 2017 by ESPN.com and No. 3 by Rivals.com. He committed to Duke on September 14, 2016, shortly after being offered a scholarship by then-Duke head coach Mike Krzy

In [9]:
new_train = [x for x in indexes_dropped_train]
new_val = [x for x in indexes_dropped_val]
print(train_y)
print(type(train_y))
train_y = list(train_y)
for index, element in enumerate(new_train):
  print(index)
  train_y = train_y[:indexes_dropped_train[index]] + train_y[indexes_dropped_train[index]+1:]
  for i in range(len(indexes_dropped_train)):
    indexes_dropped_train[i] -= 1
for index, element in enumerate(new_val):
  val_y = list(val_y[:indexes_dropped_val[index]]) + list(val_y[indexes_dropped_val[index]+1:])
  for i in range(len(indexes_dropped_val)):
    indexes_dropped_train[i] -= 1
train_y = np.asarray(train_y)
val_y = np.asarray(val_y)
train_y = torch.Tensor(train_y.reshape((len(train_y), 1)))
val_y = torch.Tensor(val_y.reshape((len(val_y), 1)))
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)
print(val_x)
print(val_y)

[1 1 1 ... 1 1 1]
<class 'numpy.ndarray'>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
torch.Size([7940, 23])
torch.Size([7940, 1])
torch.Size([1985, 23])
torch.Size([1985, 1])
tensor([[0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 8.0000e+00, 9.0000e+00,
         1.0000e+01],
        [6.0000e+00, 1.9000e+01, 1.8000e+01,  ..., 2.5000e+01, 2.0000e+01,
         2.4000e+01],
        [3.1000e+01, 1.8000e+01, 3.2000e+01,  ..., 3.2000e+01, 3.3000e+01,
         4.5000e+01],
        ...,
        [4.6700e+02, 8.8300e+02, 4.0000e+00,  ..., 3.6000e+01, 4.4630e+03,
         5.1430e+03],
        [4.4620e+03, 7.6250e+03, 2.4000e+01,  ..., 1.8000e+02, 3.6000e+01,
         1.5000e+01],
        [2.3900e+02, 4.5000e+01, 3.1170e+03,  ..., 2.4000e+01, 3.3670e+03,
         2.4000e+01]])
tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])


In [10]:
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# labels,text = load_data('data.csv')
# train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.8)

# tokenizer = get_tokenizer('basic_english')

# def yield_tokens(data_iter):
#     for text in data_iter:
#         yield tokenizer(text)

# vocab = build_vocab_from_iterator(yield_tokens(train_x), specials=["<unk>"])
# vocab.set_default_index(vocab["<unk>"])

In [19]:
text_pipeline = lambda x: x
label_pipeline = lambda x: int(x)
print(type(train_x))
print(train_x.shape)
print(type(train_y))
print(train_y.shape)
# shape_x = train_x.reshape((60000, 1))
# shape_y = train_y.reshape((60000, 1))
print(type(val_x))
print(val_x.shape)
print(type(val_y))
print(val_y.shape)
train_both = np.concatenate([train_x,train_y], axis = 1)
val_both = np.concatenate([val_x,val_y], axis = 1)
print(train_both.shape)
print(type(train_both))
print(train_both)

<class 'torch.Tensor'>
torch.Size([7940, 23])
<class 'torch.Tensor'>
torch.Size([7940, 1])
<class 'torch.Tensor'>
torch.Size([1985, 23])
<class 'torch.Tensor'>
torch.Size([1985, 1])
(7940, 24)
<class 'numpy.ndarray'>
[[0.000e+00 1.000e+00 2.000e+00 ... 1.800e+01 1.900e+01 1.000e+00]
 [0.000e+00 2.000e+01 1.000e+00 ... 2.600e+01 3.500e+01 1.000e+00]
 [3.600e+01 3.700e+01 1.100e+01 ... 8.000e+00 5.100e+01 1.000e+00]
 ...
 [7.640e+02 7.591e+03 1.880e+02 ... 1.880e+02 8.000e+00 1.000e+00]
 [1.000e+00 3.600e+01 1.892e+04 ... 3.600e+01 4.217e+03 1.000e+00]
 [3.990e+02 4.000e+02 3.200e+01 ... 8.250e+03 1.000e+00 1.000e+00]]


In [20]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for entry in batch:
         _label = entry[-1] 
         _text = entry[:len(entry)-1] 
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)
batch_size_var = 10
train_loader = DataLoader(train_both, batch_size=batch_size_var, shuffle=False, collate_fn=collate_batch)
val_loader = DataLoader(train_both, batch_size=batch_size_var, shuffle=False, collate_fn=collate_batch)
trainSteps = len(train_loader.dataset) // batch_size_var
valSteps = len(val_loader.dataset) // batch_size_var
for i, batch in enumerate(train_loader):
    print(i, batch)


Streaming output truncated to the last 5000 lines.
        15373,     0,     1,    36,   987,    11,  3824, 15373,   685,    23,
         4984,  7434,    35,   340,   288,  1802,  2863,    53,    36,  1738,
           16,    36, 15374,  6680,  2164,   685,    23,  5061, 11609, 15375,
           53,  5486,    16,    30,     1,    36, 15376,  5476,    89,   112,
          484,   209,    23,   219,    56,    53,   943, 15377,    20, 15378,
          792,    20,   254,   476,    16,     8,  1738,    60,   670,    32,
         1470,  1389,   345, 15379,     7,     8,   519,    36,  1492,   689,
         8541,  1028,    11,     8,  5211,   936,  1627,    16,   191,    53,
         2339,    20,     8,  1028,     1,  4566,     8,  3553, 10965,    53]), tensor([  0,  23,  46,  69,  92, 115, 138, 161, 184, 207]))
577 (tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), tensor([   36,  8047,   877,   209,    53, 15380,  6964,  1288,     8,   159,
           16,    30,     1,    36,   615,    11,     8, 11435

In [21]:
next_, labels_, _offset = next(iter(train_loader))
print(next_)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [128]:
# Define Architecture

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv1d(10, 4000, kernel_size=3, stride=1, padding=1),
            BatchNorm1d(23),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv1d(4000, 10, kernel_size=3, stride=1, padding=1),
            BatchNorm1d(11),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(5, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x.float())
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [129]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv1d(10, 4000, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(4000, 10, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
  )
)


In [130]:
# Create train function
def train_epoch(epoch):
    running_loss = 0
    last_loss = 0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        labels, inputs, offset = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        inputs = inputs.reshape([10, 23])
        labels = labels.reshape([10, 1])
        inputs = inputs.type(torch.LongTensor)
        inputs = inputs.to(device=device,dtype=torch.float64) 
        labels = labels.to(device=device, dtype=torch.int64)   
        
        outputs = model(inputs)
        labels = labels.type(torch.LongTensor)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels.long().squeeze(1))
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            running_loss = 0
            print('  batch {} loss: {}'.format(i + 1, last_loss))
    print(f'Epoch: {epoch}, Loss: {last_loss}')
    return last_loss

In [134]:
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.
train_losses = [] 
val_losses = []
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_epoch(epoch_number)
    train_losses.append(avg_loss)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(val_loader):
        vlabels, vinputs, offset = vdata
        vinputs = vinputs.reshape([10, 23])
        vlabels = vlabels.reshape([10, 1])
        vlabels = vlabels.type(torch.LongTensor)
        vinputs = vinputs.to(device=device,dtype=torch.float64) 
        vlabels = vlabels.to(device=device, dtype=torch.int64)   
        voutputs = model(vinputs)
        vloss = criterion(voutputs, vlabels.long().squeeze(1))
        running_vloss += vloss
    avg_vloss = running_vloss /len(val_loader)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))


    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss

    epoch_number += 1


EPOCH 1:
Epoch: 0, Loss: 0
LOSS train 0 valid 5.669245001627132e-05
EPOCH 2:
Epoch: 1, Loss: 0
LOSS train 0 valid 3.367658791830763e-05
EPOCH 3:
Epoch: 2, Loss: 0
LOSS train 0 valid 2.0965333533240482e-05
EPOCH 4:
Epoch: 3, Loss: 0
LOSS train 0 valid 1.3373281944950577e-05
EPOCH 5:
Epoch: 4, Loss: 0
LOSS train 0 valid 8.736363270145375e-06
tensor(8.7364e-06, grad_fn=<DivBackward0>)


In [135]:
 # model.train()
    # tr_loss = 0
    # # getting the training set
    # print(train_x)
    # print(train_y)
    # print(train_x.shape)
    # print(train_y.shape)
    # x_train, y_train = Variable(torch.Tensor(train_x)), Variable(torch.Tensor(train_y))
    # # getting the validation set
    # x_val, y_val = Variable(val_x), Variable(val_y)
    # # converting the data into GPU format
    # if torch.cuda.is_available():
    #     x_train = x_train.cuda()
    #     y_train = y_train.cuda()
    #     x_val = x_val.cuda()
    #     y_val = y_val.cuda()

    # # clearing the Gradients of the model parameters
    # optimizer.zero_grad()
    
    # # prediction for training and validation set
    # output_train = model(x_train)
    # output_val = model(x_val)

    # # computing the training and validation loss
    # loss_train = criterion(output_train, y_train)
    # loss_val = criterion(output_val, y_val)
    # train_losses.append(loss_train)
    # val_losses.append(loss_val)

    # # computing the updated weights of all the model parameters
    # loss_train.backward()
    # optimizer.step()
    # tr_loss = loss_train.item()
    # if epoch%2 == 0:
    #     # printing the validation loss
    #     print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [51]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
  train(epoch)
print(train_losses)
print(val_losses)

NameError: ignored